<a href="https://colab.research.google.com/github/SHRISHTI1414/AIRL-CV-Intern-Assignment/blob/main/q1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
Q1: Vision Transformer on CIFAR-10
Author: Shrishti Yadav

!pip install torch torchvision timm -q

import torch, torch.nn as nn, torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import timm, time

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Running on:", device)

1. Data loading & transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_data = datasets.CIFAR10(root="./data", train=True, download=True, transform=transform)
test_data  = datasets.CIFAR10(root="./data", train=False, download=True, transform=transform)

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader  = DataLoader(test_data,  batch_size=64, shuffle=False)

 2. Model: Vision Transformer
model = timm.create_model("vit_tiny_patch16_224", pretrained=False, num_classes=10)
model.to(device)

 3. Training setup
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)
epochs = 2        # small run for testing

start = time.time()
for ep in range(epochs):
    model.train()
    total_loss = 0
    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        out = model(imgs)
        loss = criterion(out, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {ep+1}/{epochs} | Loss: {total_loss/len(train_loader):.4f}")
print("Training done in", round((time.time()-start)/60,2), "min")

4. Evaluation
model.eval()
correct, total = 0, 0
with torch.no_grad():
    for imgs, labels in test_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        out = model(imgs)
        _, pred = torch.max(out, 1)
        total += labels.size(0)
        correct += (pred == labels).sum().item()

acc = 100 * correct / total
print(f"Test Accuracy: {acc:.2f}%")

# Save for README use
with open("vit_results.txt", "w") as f:
    f.write(f"ViT CIFAR-10 accuracy: {acc:.2f}%")


Running on: cpu
